In [ ]:
#
# This script geocodes geographic names.
#
# It performs three distinct functions:
# 1- call the geocoding servers with the input data,
# 2- create geojson files with the results, 
# 3- provide a statistical analysis.
#
# 1- Geocoding
# Five geocoders are used: Nominatim (NM), Bing (BG), Google (GO), MapBox (MB) and MapQuest (MQ).
# The input data is in a csv file with the following structure: ID, GEO, LAT, LON.
# The input data is assumed to be "clean", only basic checks are performed.
# The field "NAME" contains information separated by commas with an increasing detail as in 'Uruguay, Montevideo, Buceo'.
# If a geocoding request returns an empty result a second trial is made removing the text to the right of the last comma.
# The results are saved into separate csv files (one for each geocoder).
# The record structure for the output is: ID, GEO, NAME, LAT, LON, where ID and GEO come from the input data.
# 
# 2- Geojson
# An output is created as two geojson files from all the input data points plus the results from the geocoding.
# The two files contain a FeatureCollection with:
# - one file for all the points,
# - another file for the polygons defined by the points that share the same ID.
#
# 3- Analysis
# The geographic coordinates of the points that share the same ID are analysed with some statistical results as charts.
# The variable of study is the geodesic distance between the results of the geocoding and the input coordinates.
# This variable is defined for each ID and a comparison among the geocoding servers is possible.
#
# Units: km and decimal degrees.
# Where points are represented as pairs (tuples, lists) the coordinates are as (lon, lat) NOT as (lat, lon).
#

In [ ]:
# Version log.
# 
# R0 (20200404)
# All previous partial scripts put together.
# Seems to work well.


In [ ]:
#
# 0 COMMON PARTS
#

In [ ]:
# Import modules.
import pandas as pd
from json import dumps
from math import atan2, pi
from geopy.distance import geodesic, lonlat
import matplotlib.pyplot as plt
from numpy import cumsum, histogram
from time import sleep


In [ ]:
# Directories.
RootDir = 'YOUR_ROOT_DIR_HERE'

# I/O files:
FileNameIn = RootDir + 'YOUR_INPUT_FILE_HERE.csv' # Input data

# FileNameOut specified locally.


In [ ]:
#
# 1 GEOCODING
#

In [ ]:
# Functions
def f_georead(df_i):
    """
    Loops over df_i, calls the geocoder and loads the results (if any) into df_o.
    """
    df_o = pd.DataFrame(columns = df_i.columns)

    for i in range(0, len(df_i), 1):
        try:
            geo = df_i.iloc[i, 1]
            loc = geolocator.geocode(geo)
            if loc != None:
                print ('Location (1) found at pos {0}: {1}'.format(i, geo))
                df_o.loc[len(df_o)] = [df_i.iloc[i, 0], loc.address, loc.latitude, loc.longitude, '-']
            else:
                print ('Location (1) not found at pos {0}: {1}'.format(i, geo))
                # Try by removing the last part of the location description:
                try:
                    sleep(0.5) # just to see if it improves the server's reply...
                    geo = geo[0:geo.rfind(',')]
                    loc = geolocator.geocode(geo)
                    if loc != None:
                        print ('Location (2) found at pos {0}: {1}'.format(i, geo))
                        df_o.loc[len(df_o)] = [df_i.iloc[i, 0], loc.address, loc.latitude, loc.longitude, '-']
                    else:
                        print ('Location (2) not found at pos {0}: {1}'.format(i, geo))
                except:
                    print ('No reply for location (2) at pos {0}: {1}'.format(i, geo))
        except:
            print ('No reply for location (1) at pos {0}: {1}'.format(i, geo))
    
    print ('{0} records added'.format(len(df_o)))
    
    return df_o


In [ ]:
# Read data.
data_in = pd.read_csv(FileNameIn, index_col = None, header = 0, sep = ',')

# Remove unsuitable data (records with empty GEO, if any):
for i in range (0, len(data_in), 1):
    if type(data_in.iloc[i, 1]) != str:
        data_in.iloc[i, 1] = 1
data_in.drop(data_in[data_in['GEO'] == 1 ].index , inplace=True)

# Add new column for the source:
data_in['SOURCE'] = 'R0'

# Take a look:
data_in


In [ ]:
# Dataframe with the overall results.
data_all = data_in.copy()


In [ ]:
# Bing.

# Define tag:
tag = 'BG'

# Specify geolocator:
from geopy.geocoders import Bing
geolocator = Bing(api_key = 'YOUR_API_KEY_HERE', timeout = 5)

# Geolocate the input data and produce output df:
data_geo = f_georead(data_in)

# Mark the new data:
data_geo['SOURCE'] = tag

# Add new results to overall df:
data_all = data_all.append(data_geo)
         
# Save results.
FileNameOut = RootDir + 'YOUR_INPUT_FILE_HERE ' + tag + '.csv'
data_geo.to_csv(FileNameOut)


In [ ]:
# Google.

# Define tag:
tag = 'GO'

# Specify geolocator:
from geopy.geocoders import GoogleV3
geolocator = GoogleV3(api_key = 'YOUR_API_KEY_HERE', timeout = 5)

# Geolocate the input data and produce output df:
data_geo = f_georead(data_in)

# Mark the new data:
data_geo['SOURCE'] = tag

# Add new results to overall df:
data_all = data_all.append(data_geo)
         
# Save results.
FileNameOut = RootDir + 'YOUR_INPUT_FILE_HERE ' + tag + '.csv'
data_geo.to_csv(FileNameOut)


In [ ]:
# MapBox.

# Define tag:
tag = 'MB'

# Specify geolocator:
from geopy.geocoders import MapBox
geolocator = MapBox(api_key = 'YOUR_API_KEY_HERE', timeout = 5)

# Auxiliary df:
data_geo = pd.DataFrame(columns = data_in.columns)

# Geolocate the input data and produce output df:
data_geo = f_georead(data_in)

# Mark the new data:
data_geo['SOURCE'] = tag

# Add new results to overall df:
data_all = data_all.append(data_geo)
         
# Save results.
FileNameOut = RootDir + 'YOUR_INPUT_FILE_HERE ' + tag + '.csv'
data_geo.to_csv(FileNameOut)


In [ ]:
# MapQuest.

# Define tag:
tag = 'MQ'

# Specify geolocator:
from geopy.geocoders import OpenMapQuest
geolocator = OpenMapQuest(api_key = 'YOUR_API_KEY_HERE', timeout = 5)

# Auxiliary df:
data_geo = pd.DataFrame(columns = data_in.columns)

# Geolocate the input data and produce output df:
data_geo = f_georead(data_in)

# Mark the new data:
data_geo['SOURCE'] = tag

# Add new results to overall df:
data_all = data_all.append(data_geo)
         
# Save results.
FileNameOut = RootDir + 'YOUR_INPUT_FILE_HERE ' + tag + '.csv'
data_geo.to_csv(FileNameOut)


In [ ]:
# Nominatim.
# Define tag:
tag = 'NM'

# Specify geolocator:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent = 'YOUR_AGENT_HERE', timeout = 5)

# Geolocate the input data and produce output df:
data_geo = f_georead(data_in)

# Mark the new data:
data_geo['SOURCE'] = tag

# Add new results to overall df:
data_all = data_all.append(data_geo)
         
# Save results.
FileNameOut = RootDir + 'YOUR_INPUT_FILE_HERE ' + tag + '.csv'
data_geo.to_csv(FileNameOut)


In [ ]:
#
# 2 GEOJSON
#

In [ ]:
# Functions
def f_counterclock(df):
    '''
    Receives a dataframe (with more than three rows) with columns: (XX, XX, LAT, LON).
    Returns the dataframe sorted so that the order of the points defines a counterclockwise sequence.
    The "counterclockwiseness" is defined from the barycenter of the points.
    The sorting is done by measuring the angle from the barycenter.
    '''   
    df_len = len(df) # The function is only called with df_len >= 3, so no need to check here.
    bary = [sum(df['LON'])/df_len, sum(df['LAT'])/df_len]
    df['ANG'] = 0.
    for i in range(0, df_len, 1):
        ang = atan2(df.iloc[i][2] - bary[1], df.iloc[i][3] - bary[0])
        if ang <= 0.:
            ang += 2*pi # atan2 maps [0,180]->[0,pi] and [180,360]->[-pi,0], and I want [0,360]->[0,2pi]
        df.iloc[i,4] = ang
    df.sort_values('ANG', inplace = True)
    del(df['ANG']) # I keep it for checking purposes
    return df


In [ ]:
# Create the geojson with the points.

# The dictionary providing the structure for the featurecollection:
fc = {'type': 'FeatureCollection', 'features': []}

# Create the points as features and append them to the collection:
for i in range(0, len(data_all), 1):
    coords = [data_all.iloc[i, 3], data_all.iloc[i, 2]] # (LON,LAT)
    f ={'type': 'Feature', 
            'geometry'  : {'type': 'Point', 'coordinates': coords},
            'properties': {'id': data_all.iloc[i, 0], 'source': data_all.iloc[i, 4], 'name': data_all.iloc[i, 1]}
            }
    fc['features'].append(f)


In [ ]:
# Save.
# json conversion is required because the dictionary is with '', while geojson requires ""
FileNameOut = RootDir + 'YOUR_INPUT_FILE_HERE pnt R1.geojson'
with open(FileNameOut, 'w') as output_file:
    output_file.write(dumps(fc))
    

In [ ]:
# Create the geojson polygons.

# The dictionary providing the structure for the featurecollection:
fc = {'type': 'FeatureCollection', 'features': []}

# The set of IDs:
id_set = set(data_all.ID)

# Create the points and append them:
for item in id_set:
    # An auxiliary df with the selected points:
    df_aux = data_all[data_all.ID == item].copy()
    df_len = len(df_aux)

    # Classify according to the number of points with the same ID:
    if df_len == 0:
        print ('Error: len(df_aux) = 0 for ID =', item)
        raise SystemExit('Execution stopped.')
    elif df_len == 1:
        coords = [df_aux.iloc[0, 3], df_aux.iloc[0, 2]]
        # Create the feature:
        f ={'type': 'Feature', 
            'geometry'  : {'type': 'Point', 'coordinates': coords},
            'properties': {'id': item, 'source': df_aux.iloc[0, 4], 'name': df_aux.iloc[0, 1]}
            }
        fc['features'].append(f)
    elif df_len == 2:
        ls ={'type': 'LineString', 'coordinates': []}
        for i in range(0, df_len, 1):       
            coords = [df_aux.iloc[i, 3], df_aux.iloc[i, 2]]
            ls['coordinates'].append(coords)
        # Convert into feature:
        f ={'type': 'Feature', 
                'geometry'  : ls,
                'properties': {'id': item, 'source': df_aux.iloc[i, 4]}
           }
        fc['features'].append(f)
    else: #df_len>= 3
        # It is first needed to ensure that the points define a counterclockwise sequence:
        f_counterclock(df_aux) # df_aux is now ordered
        pl ={'type': 'Polygon', 'coordinates': []}
        aux = []
        for i in range(0, df_len, 1):       
            coords = [df_aux.iloc[i, 3], df_aux.iloc[i, 2]]
            aux.append(coords)
        pl['coordinates'].append(aux) # The geometry of the polygon is a nested list!
        # Convert into feature:
        f ={'type': 'Feature', 
                'geometry'  : pl,
                'properties': {'id': item, 'source': df_aux.iloc[i, 4]}
           } 
        fc['features'].append(f)


In [ ]:
# Save
# json conversion is required because the dictionary is with '', while geojson requires ""
FileNameOut = RootDir + 'YOUR_INPUT_FILE_HERE pol R1.geojson'
with open(FileNameOut, 'w') as output_file:
    output_file.write(dumps(fc))


In [ ]:
#
# 3 ANALYSIS
#

In [ ]:
# A container for the results:
res_abs = [[], [], [], [], []]
res_d = {'BG': 0, 'GO': 1, 'MB': 2, 'MQ': 3, 'NM': 4}


In [ ]:
# Compare the geographic data.
# Scan the set:
for item in id_set:
    # An auxiliary df with the selected points:
    df_aux = data_all[data_all.ID == item].copy()
    df_len = len(df_aux)
    
    # Verify that the data from R0 is there:
    if len(df_aux[df_aux.SOURCE == 'R0']) != 1:
        print ('Error: unsuitable data from source R0 for item: {0}'.format(item))
        raise Exception ('Program stopped.')
    
    # Auxiliary variables for easier tasks:
    PN = []
    for i in range(0, df_len, 1):
        if (df_aux.iloc[i,4] == 'R0'):
            P0 = [df_aux.iloc[i,3], df_aux.iloc[i,2], df_aux.iloc[i,4]]
            P0_aux = P0[0:2]
        else:
            PN.append([df_aux.iloc[i,3], df_aux.iloc[i,2], df_aux.iloc[i,4]])
    
    # Take the distances and store the results:
    for P in PN:
        P1_aux = P[0:2]
        d = geodesic(lonlat(*P1_aux), lonlat(*P0_aux)).km
        res_abs[res_d[P[2]]].append(d)


In [ ]:
# Charts
# Colors:
color = ['darkgray', 'salmon', 'chocolate', 'turquoise', 'lawngreen']


In [ ]:
# Make the charts.
# Histogram (absolute):
fig = plt.hist(res_abs)
plt.title(' Geocoders Accuracy', loc = 'right')
plt.xlabel('Error, km')
plt.ylabel('Number')
plt.grid(True)


In [ ]:
# Absolute frequency:
for i in range(0, len(res_abs), 1):
    hist, bins = histogram(res_abs[i], bins = 250)
    plt.scatter(bins[1:], hist, color = color[i], s = 3.0)
plt.title(' Geocoders Accuracy', loc = 'right')
plt.xlabel('Error, km')
plt.ylabel('Number')
plt.grid(True)
plt.show()


In [ ]:
# Cumulated absolute frequency:
for i in range(0, len(res_abs), 1):
    hist, bins = histogram(res_abs[i], bins = 250)
    plt.scatter(bins[1:], cumsum(hist), color = color[i], s = 3.0)
plt.title(' Geocoders Accuracy', loc = 'right')
plt.xlabel('Error, km')
plt.ylabel('Cumulated Number')
plt.grid(True)
plt.show()


In [ ]:
# Cumulated relative frequency:
for i in range(0, len(res_abs), 1):
    hist, bins = histogram(res_abs[i], bins = 250)
    hist = hist/len(res_abs[i])
    plt.scatter(bins[1:], cumsum(hist), color = color[i], s = 3.0)
plt.title(' Geocoders Accuracy', loc = 'right')
plt.xlabel('Error, km')
plt.ylabel('Cumulated Frequency')
plt.grid(True)

fig = plt.gcf()
fig.savefig(RootDir + 'Geocoder Accuracy R0.png', dpi = 600)

plt.show()


In [ ]:
# Cumulated relative frequency:
for i in range(0, len(res_abs), 1):
    hist, bins = histogram(res_abs[i], bins = 250, range = (0, 1000))
    hist = hist/len(res_abs[i])
    plt.scatter(bins[1:], cumsum(hist), color = color[i], s = 3.0)
plt.title(' Geocoders Accuracy', loc = 'right')
plt.xlabel('Error, km')
plt.ylabel('Cumulated Frequency')
plt.grid(True)
plt.xlim(0, 1000)

fig = plt.gcf()
fig.savefig(RootDir + 'Geocoder Accuracy R1.png', dpi = 600)

plt.show()
